# Load packages and specify the directory

In [2]:
import numpy as np
import pandas as pd
#import numba
from scipy.misc import comb

# Clean up data

In [3]:
df = pd.read_csv('../data/text1.csv')
df.head()

,authorNum,paperNum,paperTitle,journalTitle,coauthor
0,10,1,PMES: privilege mangagement and enforcement sy...,ifip|international|federation|information|proc...,b kvande|i levinstein|k maly|m olson|r mukkama...
1,10,2,CoProcess: A Java-based Environment for Collab...,webnet,c vemuru|h syed|h abdel-wahab|k maly|m kholief...
2,10,3,A privilege management and enforcement system ...,wetice|workshop|enabling|technologies|infrastr...,b kvande|i levinstein|k maly|m olson|r mukkama...
3,10,4,The software architecture and interprocess com...,wetice|workshop|enabling|technologies|infrastr...,a youssef|c overstreet|e stoica|h abdel-wahab|...
4,10,5,Mosaic + XTV = CoReview,computer|networks|isdn|systems,a prabhu|c vemuru|h syed|h abdel-wahab|k maly|...


In [4]:
df.journalTitle = df.journalTitle.fillna('')
df.coauthor = df.coauthor.fillna('')

In [5]:
df.journalTitle = [x.split("|") for x in df.journalTitle.tolist()]
df.coauthor = [x.split("|") for x in df.coauthor.tolist()]

In [6]:
df.head()

,authorNum,paperNum,paperTitle,journalTitle,coauthor
0,10,1,PMES: privilege mangagement and enforcement sy...,"[ifip, international, federation, information,...","[b kvande, i levinstein, k maly, m olson, r mu..."
1,10,2,CoProcess: A Java-based Environment for Collab...,[webnet],"[c vemuru, h syed, h abdel-wahab, k maly, m kh..."
2,10,3,A privilege management and enforcement system ...,"[wetice, workshop, enabling, technologies, inf...","[b kvande, i levinstein, k maly, m olson, r mu..."
3,10,4,The software architecture and interprocess com...,"[wetice, workshop, enabling, technologies, inf...","[a youssef, c overstreet, e stoica, h abdel-wa..."
4,10,5,Mosaic + XTV = CoReview,"[computer, networks, isdn, systems]","[a prabhu, c vemuru, h syed, h abdel-wahab, k ..."


# Feature Engineering

In [8]:
# use partial features to test the algorithm
features = ['coauthor','journalTitle']
label = 'authorNum'
my_df = df[features]
my_df.ix[:,'label'] = df[label].tolist()
my_df.head()

,coauthor,journalTitle,label
0,"[b kvande, i levinstein, k maly, m olson, r mu...","[ifip, international, federation, information,...",10
1,"[c vemuru, h syed, h abdel-wahab, k maly, m kh...",[webnet],10
2,"[b kvande, i levinstein, k maly, m olson, r mu...","[wetice, workshop, enabling, technologies, inf...",10
3,"[a youssef, c overstreet, e stoica, h abdel-wa...","[wetice, workshop, enabling, technologies, inf...",10
4,"[a prabhu, c vemuru, h syed, h abdel-wahab, k ...","[computer, networks, isdn, systems]",10


In [397]:
'''
def p_similarity(r1,r2):
    '''
    #parameters : r1,r2 are 2 array of arrays, e.g. [['a1','a2','a3'],['t1','t2']],[['a1','a4'],['t2','t5']]
    #output: sim is the weighted similarity between r1 and r2
    '''
    k1 = np.intersect1d(r1[0],r2[0]).shape[0]
    k2 = np.intersect1d(r1[1],r2[1]).shape[0]
    sim = np.array((k1,k2))
    return sim    
'''

In [212]:
def p_sml(r1,r2):
    '''
    parameters : r1,r2 are 2 array of arrays, e.g. [['a1','a2','a3'],['t1','t2']],[['a1','a4'],['t2','t5']]
                 w is an array, the weight parameter to adjust importance of each feature, shape is (n_feature,)
    output: sim is the similarity vector between r1 and r2
    '''
    k1 = np.intersect1d(r1[0],r2[0]).shape[0]
    k2 = np.intersect1d(r1[1],r2[1]).shape[0]
    sim = np.array((k1,k2))
    return sim 

In [213]:
'''
r1,r2 = my_df.ix[:,features].values[:2,]
#r1 = df.ix[1,].values
#r2 = df.ix[1,['coauthor','journalTitle']].values
w = np.array([0.5,0.5])
%timeit p_sml(r1,r2)
'''

The slowest run took 5.65 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 56.3 µs per loop


In [214]:
#p_sml(r1,r1)

array([8, 9])

In [401]:
'''
def C_similarity(d):
    '''
    #parameters: d1 is a clusters, an array of dim nc_records*n_feature
    #output: sim is the weighted similarity within a cluster
    '''
    if len(d.shape)==1:
        d = d.reshape(1,2)
        d = np.concatenate((d,d))
    nc = d.shape[0]
    sim = 0
    for i in range(nc-1):
        for j in range((i+1),nc):
            sim += p_similarity(d[i,],d[j,])
    return (sim/(nc*(nc-1)*1.0))   
'''       

In [402]:
#d = my_df.ix[:,features].values[:289,]
#%timeit C_similarity(d)

1 loop, best of 3: 2.71 s per loop


In [403]:
#C_similarity(d)

array([ 0.02670848,  0.17471646])

In [219]:
def C_sml(d1,d2):
    '''
    parameters: d1, d2 are 2 clusters, each is an array of dim nc_records*n_feature
                w is the weight parameter, shape is (n_feature,)
    output: sim is the similarity vector between 2 clusters
    '''
    if len(d1.shape)==1:
        d1 = d1.reshape((1,d1.shape[0]))
    if len(d2.shape)==1:
        d2 = d2.reshape((1,d2.shape[0]))
    nc_1 = d1.shape[0]
    nc_2 = d2.shape[0]
    sim = 0
    for i in range(nc_1):
        for j in range(nc_2):
            sim += p_sml(d1[i,],d2[j,])
    return (sim/(nc_1*nc_2*1.0))          

In [205]:
#d1 = my_df.ix[:,features].values[:289,]
#d2 = my_df.ix[:,features].values[289:,]
#%timeit C_sml(d1,d2,w)

1 loop, best of 3: 4.98 s per loop


In [210]:
#C_sml(d1,d2)

array([ 0.00843426,  0.31251201])

In [211]:
#C_sml(r1,r2)

array([ 2.,  0.])

In [6]:
'''
# feature 1: one gram overlapping
#@numba.jit
def cmOneGram(d):
    # d is a pair of elements in a feature
    cm_w = len(set(d[0]).intersection(set(d[1])))
    return cm_w
% timeit cmOneGram(df.ix[[1,3],'coauthor'].tolist())
# how many coauthors in average does a cluster of authors share?
# how likely are 2 clusters share same journals?
# comlexity is O(N^2)
#@numba.jit
def grpCmOneGram(d,feature='coauthor',type='avg'):
    # d is a sub-dataframe contains n records
    # feature is the feature you want to caculate overlapping
    # type is one of {'avg','ratio'} rario is common number of records normalized by length of that feature(e.g. 
    #common co-author/#total co-author)
    n = d.shape[0]
    if len(d.shape)<2:
        d = d.to_frame().transpose()
        d = pd.concat([d,d])
        n = 2
    d = d[feature].tolist()
    sum_grp = 0
    for i in range(n-1):
        for j in range((i+1),n):
            this_pair = [d[i],d[j]]
            cm_w = cmOneGram(this_pair)
            sum_grp += cm_w
    avg_grp = sum_grp/(comb(n,2)*1.0)
    return avg_grp
% timeit grpCmOneGram(df)
grpCmOneGram(df)
# cluster of records to a k-dim feature
#@numba.jit
def cluster2vec(d):
    # d is a sub-dataframe contains n records
    k1 = grpCmOneGram(d,'coauthor')
    k2 = grpCmOneGram(d,'journalTitle')
    return(np.array((k1,k2)))

'''

# Modeling

In [16]:
# compare 2 partitions
## generate T0 partition matrix
#@numba.jit
def label2mat(t):
    '''
    input: t is a 1d-array of partition labels, shape is (n_records,), e.g. array([1,1,2,3])
    output: m is a 2d-array adjacency matrix of shape (n_records,n_records)
    '''
    n = t.shape[0]
    m = np.eye(n)
    for i in range((n-1)):
        for j in range((i+1),n):
            if (t[j]==t[i]):
                m[i,j] = 1
    m += (m.transpose() - np.eye(n))
    return(m)
#@numba.jit
def mat2label(m):
    '''
    input: m is a 2d-array adjacency matrix of shape (n_records,n_records)
    output: t is a 1d-array of partition labels, shape is (n_records,), e.g. array([1,1,2,3])
    '''
    n = m.shape[0]
    t = np.zeros(n)
    k = 1
    for i in range(n):
        if (t[i]==0):
            t[m[i,]==1] = k
            k += 1
    return(t)

In [17]:
#%timeit mat2label(label2mat(my_df.label.values))

10 loops, best of 3: 56.8 ms per loop


In [441]:
## true score function
def scoreTrue(t,t_star,vec=True):
    '''
    input: t,t_star are partitions of the dataframe, e.g. t = array([1,1,2,3,1])
           vec is True use label2mat, otherwise use t
    output: s is the precision of t based on ground truth t0
    '''
    if vec:
        m = label2mat(t)
        m_star = label2mat(t_star)
        n = m.shape[0]
        agree_pairs = np.sum(m==m_star)
    else:
        agree_pairs = 0
        n = t.shape[0]
        for i in range(n-1):
            for j in range((i+1),n):
                if (((t[i]==t[j])&(t_star[i]==t_star[j]))|((t[i]!=t[j])&(t_star[i]!=t_star[j]))):
                    agree_pairs += 1
    s = agree_pairs/(n*(n-1)*0.5)
    return(s)

In [442]:
#%prun -l 4 scoreTrue(my_df.label.values[:3],my_df.label.values[:3])

In [443]:
#%timeit scoreTrue(my_df.label.values[:15],my_df.label.values[:15])

10000 loops, best of 3: 159 µs per loop


In [372]:
#%timeit scoreTrue(my_df.label.values[:15],my_df.label.values[:15],False)

10000 loops, best of 3: 160 µs per loop


In [361]:
#%timeit scoreTrue(my_df.label.values[:5],my_df.label.values[:5])

The slowest run took 22.97 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 72 µs per loop


In [331]:
#%timeit scoreTrue(my_df.label.values[:5],my_df.label.values[:5],False)

The slowest run took 4.50 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 44 µs per loop


In [444]:
## how much does precision increase after merging 2 clusters compared to before
def c_scoreTrue(i,j,t,t_star):
    '''
    input: i,j are the i-th and j-th cluster label in a t-partition
           t is the current partition, e.g. array([1,1,3,4,3])
           t_star is the 
    output: s is the precision of the new partition after merging i-th and j-th clusters
    '''
    uni_t = np.unique(t)
    new_t = np.copy(t)
    ind = ((t==uni_t[j])|(t==uni_t[i]))
    new_t[t==uni_t[j]] = uni_t[i]
    sub_before = t[ind]
    sub_after = new_t[ind]
    sub_star = t_star[ind]
    if (sum(ind)<=15):
        s = (scoreTrue(sub_after,sub_star,False)-scoreTrue(sub_before,sub_star,False))
    else:
        s = (scoreTrue(sub_after,sub_star)-scoreTrue(sub_before,sub_star))
    return s

In [375]:
#%timeit c_scoreTrue(293,302,t,t_star)

1000 loops, best of 3: 1.05 ms per loop


In [258]:
def updateW(w,gw,gw_star,h=1):
    '''
    input: w is the weight parameter to be updated
           gw is the similarity vector for algorithm predicted pair of clusters
           gw_star is the similarity vector for true pair of clusters
           h is the threshold for weighted similarity of our wrong choice of merged clusters
    '''
    alpha = 0.001
    while((gw_star.dot(w)<gw.dot(w)+1)|(gw.dot(w)>h)):
        w += (gw_star - gw)*alpha
    return w

In [255]:
'''
i = 302
j = 293
t = np.array([i for i in range(my_df.shape[0])])
X = my_df.ix[:,features].values
uni_t = np.unique(t)
gw = C_sml(X[t==uni_t[i],],X[t==uni_t[j],])
gw_star = C_sml(X[t==uni_t[0],],X[t==uni_t[1],])
'''

In [261]:
#print(gw_star.dot(w))
#print(gw.dot(w))

1.0
0.0


In [259]:
#updateW(w,gw,gw_star)

array([ 0.5,  0.5])

In [450]:
# select a nearest pair of clusters and merge them
def predMerge(t,X,w):
    '''
    input: t is a 1d-array of partition labels, shape is (n_record,)
           X is a 2d-array of records values, not including label, shape is (n_records, n_feature)
           w is a 1d-array of importance weight, shape is (n_feature,)
           t_star is a 1d-array of true partition labels, shape is (n_record,)
    output: best_t is the new partition which maximize the weighted average similarity between each pair of clusters
    '''
    uni_t = np.unique(t)
    n_c = uni_t.shape[0]
    #best_t = t
    best_ind = [-1,-1]
    max_score = -1
    gw = np.zeros(w.shape)
    for i in range(n_c-1):
        for j in range((i+1),n_c):
            this_sim = C_sml(X[t==uni_t[i],],X[t==uni_t[j],])
            this_score = this_sim.dot(w)
            if (this_score>max_score):
                max_score = this_score
                best_ind = [i,j]
                gw = this_sim
                #new_t = (t[t==uni_t[best_ind[1]]]=uni_t[best_ind[0]])
                #if (scoreTrue(new_t,t_star)<scoreTrue(t,t_star)):
                #    pass #update w
    new_t = np.copy(t)
    new_t[t==uni_t[best_ind[1]]] = uni_t[best_ind[0]]
    #return(new_t)
    return (best_ind,new_t,gw)

In [411]:
#t0 = np.array([i for i in range(my_df.shape[0])])
#X = my_df.ix[:,features].values
#%timeit predMerge(t0,X,w)

1 loop, best of 3: 13.8 s per loop


In [451]:
def createTree(t0,X,w,R):
    '''
    input: t0 is the initial partition of shape (n_record,)
           X is the 2-d array of record features, n_record*n_feature
           w is the weight of shape (n_feature,)
           R is the number of partitions in a sequence
    output: T is a sequence of  partitions, T_merge is the predicted merge clusters index, G_W is a sequence 
    of gradience at w
    '''
    T = [t0]
    T_merge = []
    G_W = []
    for r in range(R):
        ind,t,gw = predMerge(T[r],X,w)
        T.append(t)
        T_merge.append(ind)
        G_W.append(gw)
        print(r)
    return (T,T_merge,G_W)

In [415]:
#T = createTree(t0,X,w,3)

0
1
2


In [420]:
#np.unique(T[1]).shape

(576,)

In [449]:
def trueMerge(t,X,t_star,best_ind,gw):
    '''
    input: t is the partition before merge
           X is the data 2d-array
           t_star is the ground truth partition
           best_ind is the predicted best merge indexes
           gw is the predicted gradience
    output: best_ind is updated to the best merge indexes under true score, gw_star is the gradience 
    of true score at w
    '''
    uni_t = np.unique(t)
    n_c = uni_t.shape[0]
    max_score = c_scoreTrue(best_ind[0],best_ind[1],t,t_star)
    gw_star = np.copy(gw)
    #max_score = scoreTrue(t_1,t_star)
    #best_t = np.copy(t)
    #best_ind = [-1,-1]
    #max_score = -1
    for i in range(n_c-1):
        for j in range((i+1),n_c):
            this_sim = C_sml(X[t==uni_t[i],],X[t==uni_t[j],])
            #this_t = np.copy(t)
            #this_t[t==uni_t[j]] = uni_t[i]
            this_score = c_scoreTrue(i,j,t,t_star)
            #this_score = scoreTrue(this_t,t_star)
            if (this_score>max_score):
                max_score = this_score
                best_ind = [i,j]
                gw_star = this_sim
                #best_t = this_t
                break
    return (best_ind,gw_star)
    #return best_t

In [448]:
#trueMerge(T[0],X,t_star,[293,302])

[293, 302]

In [354]:
#% prun -l 4 trueMerge(t,X,t_star,[293,302])

In [409]:
'''
# update parameters
## when an error occurs
# select a nearest pair of clusters and merge them
def inner_predMerge(t,X,w):
    '''
    '''
    input: t is a 1d-array of partition labels, shape is (n_record,)
           X is a 2d-array of records values, not including label, shape is (n_records, n_feature)
           w is a 1d-array of importance weight, shape is (n_feature,)
           t_star is a 1d-array of true partition labels, shape is (n_record,)
    output: best_t is the new partition which maximize the weighted average similarity between each pair of clusters
    '''
    '''
    uni_t = np.unique(t)
    n_c = uni_t.shape[0]
    #best_t = t
    best_ind = [-1,-1]
    max_score = -1
    for i in range(n_c):
        for j in range((i+1),n_c):
            this_sim = C_similarity(X[((t==uni_t[i])|(t==uni_t[j])),])
            this_score = this_sim.dot(w)
            if (this_score>max_score):
                max_score = this_score
                best_ind = [i,j]
                #new_t = (t[t==uni_t[best_ind[1]]]=uni_t[best_ind[0]])
                #if (scoreTrue(new_t,t_star)<scoreTrue(t,t_star)):
                #    pass #update w
    new_t = np.copy(t)
    new_t[t==uni_t[best_ind[1]]] = uni_t[best_ind[0]]
    #return(new_t)
    return (best_ind,new_t)
'''

In [408]:
#%timeit inner_predMerge(t0,X,w)

1 loop, best of 3: 15.4 s per loop


In [453]:
# training
# time complexity seems too high...
def train_errorDriven(my_df,features,n_run=100,R=500,th=0.00001):
    '''
    input: my_df the cleaned-up dataframe
           features is a list of features to be caculated
           n_run is the number of iterations
           R is the deepth of partition tree
           th is the converge threshold
    output: w0 is the best weight for similarity
    '''
    X = my_df.ix[:,features].values
    t_star = my_df['label'].values
    t0 = np.array([i for i in range(t_star.shape[0])])
    w0 = np.random.uniform(size=(1,len(features)))

    for i in range(n_run):
        w = np.copy(w0)
        T,T_merge,G_W = createTree(t0,X,w,R)
        for i in range(len(T)-1):
            true_merge,gw_star = trueMerge(T[i],X,t_star,T_merge[i],G_W[i])
            if (true_merge!=T_merge[i]):
                w0 = updateW(w,G_W[i],gw_star)
                break
            
        if ((w-w0)^2<th):
            break
    return w0